# fMRI-02 Exercises

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook', font_scale=1.5)
%matplotlib inline

Today's demonstration will be in two parts. In the first section, we will show you how to generate the predicted BOLD signal for analysis of a block design experiment. In the second section, we will demonstrate from first principles why the optimal length of a block in a block design task is approximately 16 seconds.

## Part 1: Generating the predicted BOLD signal

In this first section, we will generate the predicted BOLD signal for a simple block design experiment. In this hypothetical experiment, we present to an participant alternating blocks of a [visual checkerboard](https://www.youtube.com/watch?v=xEd1h_lz4rA) (warning: flashing lights) and an empty black background, each presented for 16 seconds a time. 

A participant views six total blocks (i.e. 6 checkerboard presentations, 6 background presentations). If we were to image visual cortex, we would expect robust neural activity during the presentation of the checkerboard and baseline activity otherwise. 

To generate the expected neural activity, and its corresponding BOLD signal, we will complete the following steps:
1. Define the (super-sampled) experiment times.
2. Generate the neural "boxcars".
3. Define the hemodynamic response function (HRF).
4. Convolve the boxcar timeseries with the HRF.
5. Downsample expected BOLD timeseries.

### [1] Define (super-sampled) times
Here we define the timing of the experiment. Importantly we first define the experiment in a "super-sampled" space, or we act like we have acquired far more data points than we actually have. We do this for several reasons. First, it functions to reduce the noisiness of our convolved regressors (more on that in a minute). Second, it allows us to model events that occur between TRs.

### [2] Generate boxcars
Here we define a "boxcar" timeseries. In this step we make a binary timeseries (comprised of 0s and 1s), where 0s represent neuronal silence and 1s represent neuronal activity. Essentially, we initialize a timeseries that is as long as the number of times defined above, where the value of the timeseries is 1 if we expect the neurons to be active in that minute, and 0 otherwise. The term boxcar comes from the boxy pattern that results from this process.

### [3] Define the HRF
In this step, we define the expected shape of the HRF. Following convention, we will use the **SPM HRF**. 

### [4] Convolution
Convolution describes a particular mathematical operation where we use two functions to produce a third function that expresses how the shape of one is modified by the other. In this case, we convolve the boxcars with the HRF to model how we expect the BOLD signal to change in response to the neural activity

### [5] Downsampling
In this fifth and final step, we reduce the convolved timeseries to only those observations that we actually acquired.

## Part 1.5: Simple Regression
Next, let's use the predicted BOLD timeseries we just made and use it perform simple linear regression.

### [1] Load and visualize data

### [2] Construct design matrix
The design matrix is collection of timeseries we will predicted the observed data here. Here we use the timeseries we made and an intercept, i.e. a column of 1s.

### [3] Regression

## Part 2: fMRI detection power
In the second part of this demonstration, we will explore the measure of fMRI detection power. We will define detection power as the ability to detect nonzero changes in functional activity. Detection power lets us know how well suited our experiment design is for measuring differences in the BOLD signal from baseline or between two conditions.

### [1] Load and visualize example designs

To get started, let's look at four example block design experiments.

### [2] Defining detection power

[Liu & Frank (2004)](https://www.sciencedirect.com/science/article/pii/S1053811903005779) provided a formal definition of detection power:

$$ R_{tot} = \frac{1}{ \frac{1}{N} \sum_{i \leq j} \frac{1}{R_{ij}} } $$

Put another way, the total detection power of our experimental design, $R_{tot}$, is the inverse of the average detection power of the contrasts of interest, $R_{ij}$. Here, a contrast refers to a particular statistical difference we might calculate. For example, we might ask whether a particular condition shows a BOLD signal change different than zero; alternately, we might ask whether there is a difference in BOLD signal change between two conditions.

We define the detection power of a particular contrast as:

$$ R_{ij} = \frac{ \left[ D_{ij}(X^T X)^{-1}D_{ij} \right]^{-1} }{h^Th} $$

Now, this may look daunting but it is actually fairly simple:

- $X$ is the design matrix, i.e. the matrix of regressors from above.
- $h$ is the assumed hemodynamic response function.
- $D$ is a contrast vector, corresponding to the main effects, $[1,0], [0,1]$ and pairwise contrasts, $[1,-1]$. 

Of these values, the most important is $X^TX$, or the [Fisher information matrix](https://en.wikipedia.org/wiki/Fisher_information), which has some important properties that we will not go into here. For the present purposes, the important thing to know to know is that an optimal design will have large values along the diagonal of this matrix, and small values in its off-diagonals. Why is this so?

Consider again the design matrix, $X$. It is an $[N,K]$ matrix where $N$ is the number of time points and $K$ is the number of conditions. Therefore, $X^TX$ simply returns a $[K,K]$ matrix. The diagonals of this matrix are the dot product of the regressors with themselves. As such, it is apparent when the diagonals of the Fisher information matrix are large: when the regressors themselves, on average, deviate from zero. When the estimated BOLD response differs greatly from zero, we should expect that nonzero changes in the observed BOLD signal will be easier to detect. Hence, a design matrix with larger diagonal elements in its corresponding Fisher information matrix is more optimal.

The converse is true for the off-diagonals. The off-diagonals of the Fisher information matrix are computed from the pairwise dot product of the columns of the design matrix. As such, the off-diagonals are large when, on average, the estimated BOLD signal for two conditions deviate from zero in the same direction at the same time. If we want to be able to resolve differences between conditions, then we want the estimated BOLD signal of two conditions to be orthogonal. Hence, a design matrix with smaller off-diagonal elements in its corresponding Fisher information matrix is more optimal.

The Fisher information matrices of our four designs are presented below.

### [3] Compute and plot detection power

### [4] Re-estimating detection power assuming drifts
Why does the 4th design have such high detection power despite the conventional wisdom? The answer has to do with artifactual noise observed in fMRI data. Unfortunately, fMRI data is often contaminated by low frequency drifts. By the same logic as above, a design is robust when it is nearly orthogonal to the nuisance terms. If the on-off cycle of stimuli blocks is too slow, then it may overlap with the low-frequency drifts (i.e. off-diagonal terms) thereby reducing overall detection power. We can simulate this effect by including Legendre polynomials (up to order 2) as nuisance regressors in the design matrix.